In [1]:
#Pipeline a realizar
#1. Extracción de los datos, se obtienen los datos de la base de datos
#2. Limpieza de los datos, se eliminan los datos que no son necesarios
#3. Procesamiento de campos nulos
#4. Procesamiento para estandarizar datos categoricos
#5. Procesamiento para estandarizar datos numericos
#6. Procesamiento para estandarizar datos temporales
#7. Separación entre entrenamiento y prueba
#8. Entrenamiento de un modelo para predecir puntajes de inglés, matemáticas, critica y ciudadana
#9. Evaluación del modelo

# 1. Extracción de datos

In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd

icfes_path = "../datasets/SABERTYT20162.csv"
df = pd.read_csv(icfes_path, sep=";", encoding="latin1")
print(df.shape)

(53040, 104)


# 2. Limpieza de los datos, se eliminan los datos que no son necesarios

In [3]:
df = df.loc[~df["ESTU_ESTADO"].str.contains("VALIDEZ OFICINA JURÍDICA")] # Solo trabajar con pruebas ya validas
df = df.loc[df["ESTU_NACIONALIDAD"].str.contains("1 COLOMBIA")] # Solo trabajar con estudiantes colombianos
df.drop(["ESTU_ESTADO", "ESTU_NACIONALIDAD"], axis=1, inplace=True) # Eliminar columnas que no aportan información

columns_of_cod_to_drop = df.filter(regex='COD|DESEM|PNAL|PGREF|PRESENTACION', axis=1)
df.drop(columns=columns_of_cod_to_drop.columns, inplace=True)

columns_to_drop = ["ESTU_TIPODOCUMENTO", "PERIODO", "ESTU_CONSECUTIVO", "ESTU_ESTUDIANTE",
                "ESTU_ETNIA", "ESTU_COLE_TERMINO", "ESTU_VALOR_MATRICULAUNIVER", "ESTU_PAGO_MATRICULA_PADRES", 
                "ESTU_PAGO_MATRICULA_CREDITO", "ESTU_PAGO_MATRICULA_PROPIO", "ESTU_PAGO_MATRICULA_BECA",
                "ESTU_CURSO_DOCENTESIES", "ESTU_CURSO_IES_APOYOEXTERNO", "ESTU_CURSO_IESEXTERNA",
                "ESTU_ACTIVIDAD_REFUERZOAREA", "ESTU_ACTI_REFUERZOGENERICAS", "ESTU_COMO_CAPACITOEXAMEN",
                "ESTU_SEMESTRE_CURSA", "ESTU_TIPO_REMUNERACION", "ESTU_SIMULACRO_TIPOICFES",
                "ESTU_OTRO_PREGRADO", "ESTU_UN_POSTGRADO", "ESTU_PREGRADO_EXAM_SBPRO",
                "ESTU_CURSO_NOPREGRADO", "ESTU_OTROCOLE_TERMINO", "ESTU_SNIES_PRGMACADEMICO",
                "ESTU_ACTIVIDAD_REFUERZOAREA", "ESTU_ACTI_REFUERZOGENERICAS", "NSE", "INSE"]

df.drop(columns=columns_to_drop, inplace=True)

df["ESTU_FECHANACIMIENTO"] = pd.to_datetime(df["ESTU_FECHANACIMIENTO"], format="%d/%m/%Y")
rows_to_remove = df.loc[df["ESTU_FECHANACIMIENTO"] > "2004-01-01"].index
df.drop(rows_to_remove, inplace=True)
print(df.shape)

(52595, 52)


# 3. Procesamiento de campos nulos

In [4]:
import numpy as np

columns_with_na = df.isna().sum().sort_values(ascending=False)
print(f"Cantidad de columnas con NaN: {len(columns_with_na[columns_with_na > 0])}")

df["ESTU_TIENE_ETNIA"].fillna("NO", inplace=True)
print(f"Campos vacios ESTU_TIENE ETNIA: {df['ESTU_TIENE_ETNIA'].isna().sum()}")

columns_dissabilities = list(df.filter(regex='LIMITA', axis=1).columns)
for col in columns_dissabilities:
    df[col].replace(np.nan, False, inplace=True)
    df[col].replace("x", True, inplace=True)
print(df.filter(regex='LIMITA', axis=1).isna().sum().sort_values(ascending=False))

columns_with_na = df.isna().sum().sort_values(ascending=False)
columns_with_na = columns_with_na[columns_with_na > 0].index.tolist()
print(f"Columnas que faltan por rellenar {len(columns_with_na)}")

for column in columns_with_na:
    if df[column].dtype == 'object':
        df[column].fillna(df[column].mode()[0], inplace=True)
    elif df[column].dtype in ['float64', 'int64']:
        df[column].fillna(df[column].mean(), inplace=True)

columns_with_na = df.isna().sum().sort_values(ascending=False)
columns_with_na = columns_with_na[columns_with_na > 0].index.tolist()
print(f"Columnas que faltan por rellenar {len(columns_with_na)}")

Cantidad de columnas con NaN: 35
Campos vacios ESTU_TIENE ETNIA: 0
ESTU_LIMITA_MOTRIZ               0
ESTU_LIMITA_INVIDENTE            0
ESTU_LIMITA_CONDICIONESPECIAL    0
ESTU_LIMITA_SORDO                0
ESTU_LIMITA_AUTISMO              0
dtype: int64
Columnas que faltan por rellenar 29
Columnas que faltan por rellenar 1


In [5]:
columns_with_na

['ESTU_FECHANACIMIENTO']

# 4. Procesamiento para estandarizar datos categoricos

In [6]:
from sklearn.preprocessing import LabelEncoder


df["FAMI_NUM_PERSONASACARGO"].unique()
df["FAMI_NUM_PERSONASACARGO"] = df["FAMI_NUM_PERSONASACARGO"].replace("12 O MÁS", 12).astype(int)

cat_attribs = df.select_dtypes(include='object').columns.tolist()

label_encoder = LabelEncoder()
mapping_info = []

for col in cat_attribs:
    df[col] = label_encoder.fit_transform(df[col])

    original_classes = label_encoder.classes_
    col_mapping = [(original, encoded) for original, encoded in zip(original_classes, range(len(original_classes)))]
    mapping_info.append({'Columna': col, 'Mapeo': col_mapping})

# 5. Procesamiento para estandarizar datos numericos

In [7]:
float_columns = df.select_dtypes(include='float64').columns.tolist()
df[float_columns] = df[float_columns].astype(int)

int_32_columns = df.select_dtypes(include='int32').columns.tolist()
df[int_32_columns] = df[int_32_columns].astype('int64')

bool_columns = df.select_dtypes(include='bool').columns.tolist()
df[bool_columns] = df[bool_columns].astype('int64')

# 6. Procesamiento para estandarizar datos temporales

In [8]:
df["EDAD"] = (pd.to_datetime("2016-10-09") - df["ESTU_FECHANACIMIENTO"]).dt.days // 365
df.drop("ESTU_FECHANACIMIENTO", axis=1, inplace=True)

# 7. Separación entre entrenamiento y prueba

In [9]:
columns_areas = ["MOD_RAZONA_CUANTITAT_PUNT", "MOD_LECTURA_CRITICA_PUNT", "MOD_COMPETEN_CIUDADA_PUNT", "MOD_INGLES_PUNT"]

X = df.drop(columns_areas, axis=1)
Y = df[columns_areas]

from  sklearn.model_selection import train_test_split
X_train, x_test, Y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# 8. Entrenamiento de un modelo para predecir 
# puntajes de inglés, matemáticas, critica y ciudadana

In [10]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
forest_reg.fit(X_train, Y_train)

RandomForestRegressor(random_state=42)

# 9. Evaluación del modelo

In [11]:
from sklearn.metrics import mean_absolute_error

y_pred = forest_reg.predict(x_test)

y_test = np.asarray(y_test)

mae_por_materia = []

for i in range(len(y_test[0])):
    mae = mean_absolute_error([y_test[j][i] for j in range(len(y_test))], [y_pred[j][i] for j in range(len(y_pred))])
    mae_por_materia.append(mae)

for i, mae in enumerate(mae_por_materia):
    print(f"MAE para materia {Y_train.columns[i]}: {mae}")

MAE para materia MOD_RAZONA_CUANTITAT_PUNT: 15.80607662325316
MAE para materia MOD_LECTURA_CRITICA_PUNT: 16.621407928510315
MAE para materia MOD_COMPETEN_CIUDADA_PUNT: 16.62305257153722
MAE para materia MOD_INGLES_PUNT: 15.017699401083753


In [12]:
importances = forest_reg.feature_importances_

indices = np.argsort(importances)[::-1]

# Imprime el ranking de importancia de los atributos
print("Ranking de importancia de atributos:")
for f in range(X_train.shape[1]):
    print(f"{X_train.columns[indices[f]]}: {importances[indices[f]]}")

Ranking de importancia de atributos:
MOD_COMUNI_ESCRITA_PUNT: 0.11084542130535914
EDAD: 0.0597174736395324
ESTU_PRGM_ACADEMICO: 0.05770221163782689
ESTU_MCPIO_RESIDE: 0.041788490697228245
FAMI_OCUPACION_MADRE: 0.04030143403674288
FAMI_EDUCACION_PADRE: 0.039294098705028105
INST_NOMBRE_INSTITUCION: 0.03795362425461686
FAMI_EDUCACION_MADRE: 0.03648663802815901
GRUPOREFERENCIA: 0.036057420956891115
FAMI_OCUPACION_PADRE: 0.03551419496251944
ESTU_NUCLEO_PREGRADO: 0.033282892955089426
FAMI_PERSONAS_HOGAR: 0.032960957940508434
FAMI_ESTRATO_VIVIENDA: 0.032106025764989686
ESTU_NUMERO_LIBROS: 0.03146603226916474
ESTU_PRGM_MUNICIPIO: 0.029384154349860206
FAMI_CUARTOS_HOGAR: 0.024677685515120992
ESTU_DEPTO_RESIDE: 0.024162894978950366
ESTU_DEDICACION_LECTURADIARIA: 0.02348511925000834
ESTU_HORAS_SEMANATRABAJA: 0.02251700486568464
FAMI_NUM_PERSONASACARGO: 0.02015141337437247
ESTU_PRGM_DEPARTAMENTO: 0.01983883501143357
ESTU_DEDICACION_INTERNET: 0.01764278344311847
INST_ORIGEN: 0.015497699158694084
ES